In [7]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MyApp, master=local) created by __init__ at <ipython-input-1-a0aae5f98c4b>:2 

In [2]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        result=[]
        for word in words:
            word_l=word.lower()
            if word_l not in stopwords:
                result.append(word_l)
        return result
    except ValueError as e:
        return []

def bigrams(words):
    result=[]
    for w1,w2 in zip(words,words[1:]):
        result.append((w1+"_"+w2,1))
    return result

#wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16)\
#.map(parse_article)\
#.flatMap(bigrams)\
#.reduceByKey(lambda x,y:x+y)\
#.filter(lambda x: x[1]>=500)


In [4]:
stopwords = sc.textFile("/datasets/stop_words_en.txt", 16).collect()
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16)\
.map(parse_article).cache()

In [5]:
w_bigrams=wiki\
.flatMap(bigrams)\
.reduceByKey(lambda x,y:x+y)\
.filter(lambda x: x[1]>=500).cache()

In [ ]:
w_bigrams.take(2)

In [ ]:
total_bigrams=w_bigrams.reduce(lambda x:x[1]).sum()

In [25]:
w_unigrams=wiki\
.flatMap(lambda x: x)\
.map(lambda x: (x,1))\
.reduceByKey(lambda x,y:x+y)\
.sortByKey()\
.cache()
#.sortBy(lambda x: x[0])
#list_unigrams=w_unigrams.collect()

In [27]:
total_unigrams=w_unigrams.reduce(lambda x:x[1]).sum()

In [35]:
broadcastVar = sc.broadcast(w_unigrams.collectAsMap())

In [45]:
from math import log
def npmi(input_bigram):
    bigram, count_ab=input_bigram
    a,b=bigram.split('_')
    count_a=broadcastVar.value.get(a)
    count_b=broadcastVar.value.get(b)
    
    pa=float(count_a)/total_unigrams
    pb=float(count_b)/total_unigrams
    pab=float(count_ab)/total_bigrams
    #PMI(a, b) = ln (P(ab) / (P(a) * P(b))
    pmi_ab=log(pab/(pa*pb))
    #NPMI(a, b) = PMI(a, b) / -ln P(ab)
    res_npmi=pmi_ab/-log(pab)
    
    return (bigram,res_npmi)

result=w_bigrams.map(npmi).sortBy(lambda x: x[1],ascending=False).cache()

In [48]:
for res in result.take(39):
    print(res[0])

united_states
new_york
world_war
external_links
war_ii
american_actor
university_press
united_kingdom
references_external
years_later
civil_war
air_force
19th_century
american_actress
20th_century
north_american
north_america
york_city
new_zealand
american_baseball
united_nations
soviet_union
high_school
american_football
prime_minister
baseball_player
american_singer-songwriter
south_africa
notes_references
catholic_church
roman_catholic
took_place
award_best
roman_empire
references_reading
supreme_court
los_angeles
et_al
san_francisco
